## Data preprocessing

##### Copyright (C) Microsoft Corporation.  
see license file for details 

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# AZUREML_NATIVE_SHARE_DIRECTORY mapping to host dir is set by _nativeSharedDirectory_ in .compute file 

import os
try:
    amlWBSharedDir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']    
except:
    amlWBSharedDir = ''
    print('not using aml services?')
    
amlWBSharedDir

'/azureml-share/'

In [3]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [4]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [5]:
# import utlity functions

import sys, os
paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['Code',  'src'])))]
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)

[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

[None]

#### Path variables

In [6]:
# create base directories for the file path variables 
# paths are tipically container level dirs mapped to a host dir for data persistence.

prj_consts = azure_chestxray_utils.chestxray_consts()

data_base_input_dir=os.path.join(amlWBSharedDir, os.path.join(*(prj_consts.BASE_INPUT_DIR_list)))
data_base_output_dir=os.path.join(amlWBSharedDir, os.path.join(*(prj_consts.BASE_OUTPUT_DIR_list)))  

data_base_input_dir
data_base_output_dir


'/azureml-share/chestxray/data/ChestX-ray8'

'/azureml-share/chestxray/output'

In [7]:
# chest xray images are in nih_chest_xray_data_dir
nih_chest_xray_data_dir=os.path.join(data_base_input_dir, 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))
nih_chest_xray_data_dir

# check if we have all 112120 images in nih_chest_xray_data_dir
orig_images_no = !find $nih_chest_xray_data_dir -type f | wc -l
print("orig images number:{} ".format(orig_images_no))

'/azureml-share/chestxray/data/ChestX-ray8/ChestXray-NIHCC'

orig images number:['112120'] 


In [8]:
# check if we have patients file list Data_Entry_2017.csv and BBox_List_2017.csv (https://nihcc.app.box.com/v/ChestXray-NIHCC)
# blacklist.csv is genrated by data scientists with no medical background

other_data_dir=os.path.join(data_base_input_dir, os.path.join(*(prj_consts.ChestXray_OTHER_DATA_DIR_list)))
other_data_dir
# !mkdir -p {other_data_dir}

import shutil
shutil.copyfile(os.path.join(os.getcwd(), os.path.join(*(['Code','src','finding_lungs','blacklist_non_PA_AP_view.csv']))),
                os.path.join(other_data_dir,'blacklist.csv'))

!ls $other_data_dir

# data is split into train/test/validation partitions
data_partitions_dir=os.path.join(data_base_output_dir, os.path.join(*(prj_consts.DATA_PARTITIONS_DIR_list)))  
!mkdir -p {data_partitions_dir}

'/azureml-share/chestxray/data/ChestX-ray8/ChestXray-NIHCC_other'

'/azureml-share/chestxray/data/ChestX-ray8/ChestXray-NIHCC_other/blacklist.csv'

BBox_List_2017.csv  Data_Entry_2017.csv  blacklist.csv


In [9]:
import pickle
import random
import re
import tqdm

import cv2
import numpy as np
import pandas as pd
import sklearn.model_selection 

#### Train/Validation/Test Data partitioning 
 - remove the images in the blacklist.csv where the image has low quality. 
 - remove the NIH bounding box patients since we will save those patients for later validation use. 
 - We will also divide data into train/valid/test dataset using a 7:1:2 ratio.

In [10]:
# remove NIH manually annotated data (groung truth with heavy pathologies, no healthy patients) 
# exclude what visusally looks like bad images to data scientists with no medical background
# todo
# This should prob be a generic function


total_patient_number = 30805
NIH_annotated_file = 'BBox_List_2017.csv' # exclude from train pathology annotated by radiologists 
manually_selected_bad_images_file = 'blacklist.csv'# exclude what viusally looks like bad images

patient_id_original = [i for i in range(1,total_patient_number + 1)]

# ignored images list is used later, since this is not a patient ID level issue
ignored_images_set = set()
with open(os.path.join(other_data_dir, manually_selected_bad_images_file), 'r') as f:
    for line in f:
        # delete the last char which is \n
        ignored_images_set.add(line[:-1])
        if int(line[:-9]) >= 30805:
            print(line[:-1])

bbox_df = pd.read_csv(os.path.join(other_data_dir, NIH_annotated_file))
bbox_patient_index_df = bbox_df['Image Index'].str.slice(3, 8)

bbox_patient_index_list = []
for index, item in bbox_patient_index_df.iteritems():
    bbox_patient_index_list.append(int(item))

patient_id = list(set(patient_id_original) - set(bbox_patient_index_list))
print("len of original patient id is", len(patient_id_original))
print("len of cleaned patient id is", len(patient_id))
print("len of unique patient id with annotated data", 
      len(list(set(bbox_patient_index_list))))
print("len of patient id with annotated data",bbox_df.shape[0])


len of original patient id is 30805
len of cleaned patient id is 30079
len of unique patient id with annotated data 726
len of patient id with annotated data 984


In [11]:
# move/copy all bbox images in test folder since these are not used for training anyway

test_images_dir=os.path.join(data_base_input_dir, 'test_images')
!mkdir -p {test_images_dir}

bbox_images_list = list(bbox_df['Image Index'])
for crt_file in tqdm.tqdm(bbox_images_list):
        source_file = os.path.join(nih_chest_xray_data_dir,crt_file)
        dest_file = os.path.join(test_images_dir,crt_file)
        if os.path.exists(dest_file):
            print('WARNING: image file '+crt_file+' already exists in '+test_images_dir)
#         print('crt_file')

        try:
            shutil.copyfile(source_file,dest_file)
        except IOError:
            print('WARNING: File '+ src_file_path + ' does not exist when copying to '+test_images_dir )

  0%|          | 0/984 [00:00<?, ?it/s]

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00013118_008.png'

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00014716_007.png'

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00029817_009.png'

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00014687_001.png'

...

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00025769_001.png'

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00016837_002.png'

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00020124_003.png'

'/azureml-share/chestxray/data/ChestX-ray8/test_images/00026920_000.png'

100%|██████████| 984/984 [00:04<00:00, 209.97it/s]


In [12]:
random.seed(0)
random.shuffle(patient_id)

print("first ten patient ids are", patient_id[:10])

# training:valid:test=7:1:2
patient_id_train = patient_id[:int(total_patient_number * 0.7)]
patient_id_valid = patient_id[int(total_patient_number * 0.7):int(total_patient_number * 0.8)]
# get the rest of the patient_id as the test set
patient_id_test = patient_id[int(total_patient_number * 0.8):]
patient_id_test.extend(bbox_patient_index_list)
patient_id_test = list(set(patient_id_test))


print("train:{} valid:{} test:{}".format(len(patient_id_train), len(patient_id_valid), len(patient_id_test)))

# test_set = test_set+left_out_patient_id
# print("train:{} valid:{} test:{}".format(len(train_set), len(valid_set), len(test_set)))

first ten patient ids are [24303, 16035, 4967, 28624, 5378, 20335, 17069, 12271, 16975, 4469]
train:21563 valid:3081 test:6161


In [13]:
# Add a few more project constants

pathologies_name_list = prj_consts.DISEASE_list
NIH_patients_and_labels_file = 'Data_Entry_2017.csv'

#### Finally do preprocessing
Save labels and partitions

In [14]:
labels_df = pd.read_csv(os.path.join(other_data_dir, NIH_patients_and_labels_file))

In [15]:
#show the label distribution

# Unique IDs frequencies can be computed using list comprehension or collections lib
# [[x,(list(crtData['fullID2'])).count(x)] for x in set(crtData['fullID2'])]
# for tallying, collections lib is faster than list comprehension
from collections import Counter
pathology_distribution = Counter(list(labels_df['Finding Labels']))

# Sort it by ID frequency (dict value)
sorted_by_freq = sorted(pathology_distribution.items(), key=lambda x: x[1], reverse=True)
len(sorted_by_freq)
sorted_by_freq[:20]
sorted_by_freq[-10:]

836

[('No Finding', 60361),
 ('Infiltration', 9547),
 ('Atelectasis', 4215),
 ('Effusion', 3955),
 ('Nodule', 2705),
 ('Pneumothorax', 2194),
 ('Mass', 2139),
 ('Effusion|Infiltration', 1603),
 ('Atelectasis|Infiltration', 1350),
 ('Consolidation', 1310),
 ('Atelectasis|Effusion', 1165),
 ('Pleural_Thickening', 1126),
 ('Cardiomegaly', 1093),
 ('Emphysema', 892),
 ('Infiltration|Nodule', 829),
 ('Atelectasis|Effusion|Infiltration', 737),
 ('Fibrosis', 727),
 ('Edema', 628),
 ('Cardiomegaly|Effusion', 484),
 ('Consolidation|Infiltration', 441)]

[('Atelectasis|Cardiomegaly|Consolidation|Effusion|Pneumothorax', 1),
 ('Effusion|Emphysema|Infiltration|Mass|Nodule', 1),
 ('Atelectasis|Edema|Effusion|Infiltration|Pleural_Thickening', 1),
 ('Atelectasis|Cardiomegaly|Consolidation|Effusion|Infiltration|Pneumonia', 1),
 ('Cardiomegaly|Effusion|Infiltration|Pneumonia', 1),
 ('Atelectasis|Consolidation|Effusion|Fibrosis|Pleural_Thickening', 1),
 ('Effusion|Nodule|Pneumothorax|Mass', 1),
 ('Effusion|Fibrosis|Nodule', 1),
 ('Consolidation|Effusion|Mass|Nodule|Pneumothorax', 1),
 ('Atelectasis|Emphysema|Fibrosis|Nodule|Pleural_Thickening|Pneumothorax', 1)]

In [16]:
# now label distribution after spliting

labels_df['Finding Labels'].str.split( '|', expand=False).str.join(sep='*').str.get_dummies(sep='*').sum()

Atelectasis           11559
Cardiomegaly           2776
Consolidation          4667
Edema                  2303
Effusion              13317
Emphysema              2516
Fibrosis               1686
Hernia                  227
Infiltration          19894
Mass                   5782
No Finding            60361
Nodule                 6331
Pleural_Thickening     3385
Pneumonia              1431
Pneumothorax           5302
dtype: int64

In [17]:
def process_data(current_df, patient_ids):
    image_name_index = []
    image_labels = {}
    for individual_patient in tqdm.tqdm(patient_ids):
        for _, row in current_df[current_df['Patient ID'] == individual_patient].iterrows():
            processed_image_name = row['Image Index']
            if processed_image_name in ignored_images_set:
                pass
            else:
                image_name_index.append(processed_image_name)
                image_labels[processed_image_name] = np.zeros(14, dtype=np.uint8)
                for disease_index, ele in enumerate(pathologies_name_list):
                    if re.search(ele, row['Finding Labels'], re.IGNORECASE):
                        image_labels[processed_image_name][disease_index] = 1
                    else:
                        # redundant code but just to make it more readable
                        image_labels[processed_image_name][disease_index] = 0
                # print("processed", row['Image Index'])
    return image_name_index, image_labels


In [18]:
# # create and save train/test/validation partitions list

train_data_index, train_labels = process_data(labels_df, patient_id_train)
valid_data_index, valid_labels = process_data(labels_df, patient_id_valid)
test_data_index, test_labels = process_data(labels_df, patient_id_test)

print("train, valid, test image number is:", len(train_data_index), len(valid_data_index), len(test_data_index))

# save the data
labels_all = {}
labels_all.update(train_labels)
labels_all.update(valid_labels)
labels_all.update(test_labels)

partition_dict = {'train': train_data_index, 'test': test_data_index, 'valid': valid_data_index}

with open(os.path.join(data_partitions_dir,'labels14_unormalized_cleaned.pickle'), 'wb') as f:
    pickle.dump(labels_all, f)

with open(os.path.join(data_partitions_dir,'partition14_unormalized_cleaned.pickle'), 'wb') as f:
    pickle.dump(partition_dict, f)
    
# also save the patient id partitions for pytorch training    
with open(os.path.join(data_partitions_dir,'train_test_valid_data_partitions.pickle'), 'wb') as f:
    pickle.dump([patient_id_train,patient_id_valid,
                 patient_id_test,
                list(set(bbox_patient_index_list))], f)    


100%|██████████| 6161/6161 [00:14<00:00, 436.44it/s]


train, valid, test image number is: 69190 9594 33281


In [19]:
# sanity check, see train labels

type(train_labels)
{k: train_labels[k] for k in list(train_labels)[:5]}

dict

{'00018080_007.png': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 '00021033_000.png': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 '00021362_000.png': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 '00024930_000.png': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8),
 '00030379_002.png': array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)}

In [20]:
# jupyter nbconvert --to html .\Code\02_Model\000_preprocess.ipynb